In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
my_path = "drive/My Drive/data/"

In [ ]:
if False:
  data = {}
  with open('sentiment-analysis/data/train.csv', newline='') as csvfile:
    for i in csv.reader(csvfile):
      data.setdefault(i[1], list()).append(i[0])
  data.pop('label')
  processed = dict((i, list(map(f, j))) for i, j in data.items())
  f = lambda x: ' '.join((i[1:-1] if i[0] == i[-1] == "'" else i) for i in ''.join((i if (i == "'" or i not in string.punctuation) else ' ') for i in x).split())
  open('amina.train', 'w').write('\n'.join('\n'.join('__label__%d , %s' % (random.choice((1, 3)) if int(label) else random.choice((2, 4)), i) for i in subdata) for label, subdata in processed.items()))

if False:
  data = {}
  with open('sentiment-analysis/data/test.csv', newline='') as csvfile:
    for i in csv.reader(csvfile):
      data.setdefault(i[1], list()).append(i[0])
  data.pop('label')
  processed = dict((i, list(map(f, j))) for i, j in data.items())
  f = lambda x: ' '.join((i[1:-1] if i[0] == i[-1] == "'" else i) for i in ''.join((i if (i == "'" or i not in string.punctuation) else ' ') for i in x).split())
  open('amina.test', 'w').write('\n'.join('\n'.join('__label__%d , %s' % (random.choice((1, 3)) if int(label) else random.choice((2, 4)), i) for i in subdata) for label, subdata in processed.items()))







 - Only encoder part of Transformer model is used for classification1
 - No residual connection, no layer normalization
 - No need for masking
 - Multihead attention and positionwise feedforward network to extract features
 - Then, linear layer to get logits
 



In [ ]:
import os
import sys
import torch
from torch.nn import functional as F
import numpy as np
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
import pandas as pd
import torch
from torchtext import data

# pour la reproductibilité
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

MAX_SEN_LEN = 200
TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, fix_length= MAX_SEN_LEN)
LABEL = data.Field(sequential=False, use_vocab=False)

train_data, valid_data, test_data = data.TabularDataset.splits(
        path=my_path, train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Nombre de tokens unique dans le TEXT: {len(TEXT.vocab)}") 
print(f"Nombre unique de LABEL: {len(LABEL.vocab)}")

BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)


In [ ]:
# train_utils.py

import torch
from torch import nn
from torch.autograd import Variable
import copy
import math

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Embeddings(nn.Module):
    '''
    Usual Embedding layer with weights multiplied by sqrt(d_model)
    '''
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
    
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(torch.as_tensor(position.numpy() * div_term.unsqueeze(0).numpy()))
        pe[:, 1::2] = torch.cos(torch.as_tensor(position.numpy() * div_term.unsqueeze(0).numpy()))#torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layer normalization module."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
    
class SublayerOutput(nn.Module):
    '''
    A residual connection followed by a layer norm.
    '''
    def __init__(self, size, dropout):
        super(SublayerOutput, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Positionwise feed-forward network."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        "Implements FFN equation."
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

## Encoder


In [ ]:
class Encoder(nn.Module):
    '''
    Transformer Encoder
    
    It is a stack of N layers.
    '''
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)
    
class EncoderLayer(nn.Module):
    '''
    An encoder layer
    
    Made up of self-attention and a feed forward layer.
    Each of these sublayers have residual and layer norm, implemented by SublayerOutput.
    '''
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer_output = clones(SublayerOutput(size, dropout), 2)
        self.size = size

    def forward(self, x, mask=None):
        "Transformer Encoder"
        x = self.sublayer_output[0](x, lambda x: self.self_attn(x, x, x, mask)) # Encoder self-attention
        return self.sublayer_output[1](x, self.feed_forward)

## Attention

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    "Implementation of Scaled dot product attention"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Multi-head attention"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [ ]:
import numpy as np
import torch.optim as optim
from utils import *
from torch.nn import functional as F

class Transformer(nn.Module):
    def __init__(self, N, d_model, d_ff, H, dropout,output_size,batch_size, vocab):
        super(Transformer, self).__init__()
        self.N = N

        self.d_model = d_model
        self.d_ff =  d_ff
        self.h = h
        self.dropout = dropout
        self.output_size = output_size
        self.batch_size = batch_size  
        self.vocab = vocab

        h, N, dropout = self.h, self.N, self.dropout
        d_model, d_ff = self.d_model, self.d_ff
        
        attn = MultiHeadedAttention(h, d_model)
        ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        position = PositionalEncoding(d_model, dropout)
        
        self.encoder = Encoder(EncoderLayer(d_model, deepcopy(attn), deepcopy(ff), dropout), N)
        self.src_embed = nn.Sequential(Embeddings(d_model, self.vocab), deepcopy(position)) #Embeddings followed by PE

        # Fully-Connected Layer
        self.fc = nn.Linear(
            self.config.d_model,
            self.config.output_size
        )
        
        # Softmax non-linearity
        self.softmax = nn.Softmax()

    def forward(self, x):
        embedded_sents = self.src_embed(x.permute(1,0)) # shape = (batch_size, sen_len, d_model)
        encoded_sents = self.encoder(embedded_sents)
        
        # Convert input to (batch_size, d_model) for linear layer
        final_feature_map = encoded_sents[:,-1,:]
        final_out = self.fc(final_feature_map)
        return self.softmax(final_out)
    

In [ ]:

N = 1 #6 in Transformer Paper
D_MODEL = 256 #512 in Transformer Paper
D_FF = 512 #2048 in Transformer Paper
H = 8
DROPOUT = 0.1
OUTPUT_SIZE = 2




model = Transformer(N, D_MODEL,D_FF, H, DROPOUT,OUTPUT_SIZE,BATCH_SIZE, len(TEXT.vocab))
  if torch.cuda.is_available():
    model.cuda()

optimizer = optim.Adam(self.parameters())
criterion = F.cross_entropy 

In [ ]:
def binary_accuracy(self, preds, y):
  """
  Retourne l'accuracy par batch
  """
  #arrondi la prédiction à l'entier le plus proche
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float() 
  acc = correct.sum() / len(correct)
  return acc

In [ ]:
def train_model(model, iterator):
        
        epoch_loss = 0
        epoch_acc = 0


        self.train()
    
        for batch in iterator:
        
            optimizer.zero_grad()
            text = batch.text[0]
            
            target = batch.label 

            if (text.size()[1] is not self.config.batch_size):
                continue
            predictions = model(text)
     
            target = torch.autograd.Variable(target).long()
            loss = criterion(predictions, target)
            #target = [batch_size]
            #predictions = [batch_size, output_size]
            
            pred = torch.max(predictions, 1)[1].view(target.size()).data
            #pred = [batch_size]
            acc = binary_accuracy(pred.float(), target)

            loss.backward()
        
            optimizer.step()
        
            epoch_loss += loss.item()
            epoch_acc += acc.item()
 
        return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator):
      
        epoch_loss = 0
        epoch_acc = 0
        model.eval()
      
        with torch.no_grad():
      
            for batch in iterator:

                text = batch.text[0]
                if (text.size()[1] is not batch_size):# One of the batch returned by BucketIterator has length different than 32.
                    continue
                target = batch.label 
                predictions = model(text)
     
                target = torch.autograd.Variable(target).long()
                loss = criterion(predictions, target )
                pred = torch.max(predictions, 1)[1].view(target.size()).data
            
                acc = binary_accuracy(pred.float(), target)
        
                epoch_loss += loss.item()
                epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)     

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator)
    valid_loss, valid_acc = evaluate(model, valid_iterator)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



In [ ]:
test_loss, test_acc = evaluate(model, test_iterator)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%| Test Recall: {test_rec*100:.2f}%  | Test F1: {test_f1*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    length_tensor = torch.LongTensor(length)
    prediction = model(tensor)
    out = F.softmax(prediction, 1)
    if (torch.argmax(out[0]) == 0):
      print ("Sentiment: Positive")
    else:
      print ("Sentiment: Negative")

predict_sentiment(model, "This film is terrible")


predict_sentiment(model, "This film is great amazing good ")

